In [1]:
# this is to run one of hal's model using my code

In [2]:
# let's use the one he has in the email

```
Hi Yimeng,

My most standard model types are here on the cluster:

/user_data/hrockwel/gaya_data/modeling/saved_models/transfer_learning/near_surround_batch_tang_it6_ds2__densenet121_block10_chan32_ff1_r3_dil1_gr1_ds1__lr0.003_scale0.001_e300_sd0

The last part of that, with "sd0", refers to the model seed. Of this type I have 40 different random seeds, but due to a labeling mistake earlier, the first 4 were overwritten by a model with slightly different training. You can just ignore that, since it doesn't make much difference, or if you really want, only look at "sd4" through "sd39", though you will have to modify the way "sd" is used to index into the accumulated "all_diffs". (I've since fixed this issue locally, sorry about that).

In terms of running "model_orientation_tuning.py", you can just replace the stuff with "MODELING_DIR" and the key with the above string, just substituting in the seed for each iteration of the loop over seeds. 

There are some parameters that need to be specified when the model is loaded too, not just in that string. I think these should all be correct in the defaults of the script on Github. If not, send my an email.

Also note that you'll need to get the orientation stimuli by re-running "download_data.sh" in your repo. Alternatively, you can just run the last line of it manually to get the orientation stimuli.

Now for the explanation of the script. The first set of commented-out code for figures, on line 116 on Github, plots orientation tuning curves for the set of channels specified. This was used as a sanity check partway through, to make sure channels actually had orientation tuning, and to eyeball whether they seemed to correspond to the kernels. Originally it showed the corresponding 3x3 kernels on top of the images, but I changed it later to plot those separately. Uncommenting the lines commented with "#" should probably revert back to the original behavior. 

The second set of commented figure code, at line 220, plots the recurrent kernel's projection onto each orientation, in descending order of tuning preference for that orientation. This was a way of looking more closely for an association field for each individual channel, before I came up with a quantitative metric. What you would expect to see from an association field is the bars decreasing in height from left to right. 

The final two sets of figure code, at 247 and 257, are averages over all channels in a model. The first figure shows the overall extent of orientation tuning in the model, basically summarizing the plots of line 116. The second figure plots the preferred orientation kernel weighting on the y-axis versus its orthogonal on the x-axis, and so would see most points above the y=x line for a model with an association field. 

Finally, there are the numerical results. These are reported first for each model/seed at line 242, then averaged over all models at line 269. The "association field weighting" is computed by the projection of the kernel onto the preferred tuning direction, minus the projection onto the orthogonal of that direction. In both cases, we print the average value, and its standard error (across all channels, disregarding their original models). For a model with an association field, we expect to see a positive value at least 3-4 standard errors away from zero. 

For my particular models above, the results are usually not strong enough to be reliably positive for a single model, so I primarily pay attention to the very last one printed, which is averaged over all seeds. 

The final lines are from a single experiment I did, where I checked if there was a correlation between test-set performance and association field weighting among different seeds (this is why I trained 40 of this type). There is not, so there's no particular reason to care about or include that code. 

Let me know if you have any other questions. The main things you would want to modify for your own use should be pretty clear, but I'll highlight a few: around line 55, line 72 (normalization), line 88 (activation extraction), and line 110 (weight extraction). 

Best,
Hal
```

In [3]:
from analysis import FIG_DIR
from analysis.data_utils import get_neural_data, spike_counts, \
        trial_average, DATA_DIR
from modeling.data_utils import get_images, torchvision_normalize, \
        train_val_test_split
from modeling import MODELING_DIR
from modeling.models.extended_transfer.trans_densenets import intermediate_size, TransferDensenet
from modeling.models.extended_transfer.near_surround import NearSurroundTransfer

In [4]:
# data params
DATASET = 'tang'
CORR_THRESHOLD = 0.7
START = 530
END = 1130
WINDOW_SIZE = 100
SORT = 'batch'
DOWNSAMPLE = 2

# model params
block = 10
net_type = 'densenet'
net_size = 121
chan = 32
ff_k = 1
r_k = 3
groups = 1
image_size = 252
n_neurons = 34
iterations = 6
dilation = 1
seeds = list(range(10))
epochs = 300
trans_out_shape = intermediate_size(net_size, block, image_size // DOWNSAMPLE)


In [5]:
import torch
import numpy as np

In [6]:
sd = 4 # 0,1,2,3 are bad, not loadable, as Hal said.
model_key = f'near_surround_batch_tang_it{iterations}_ds{DOWNSAMPLE}__{net_type}{net_size}_block{block}_chan{chan}_ff{ff_k}_r{r_k}_dil{dilation}_gr{groups}_ds1__lr0.003_scale0.001_e{epochs}_sd{sd}'
model_dir = f'{MODELING_DIR}/saved_models/transfer_learning/{model_key}/'
base_net = TransferDensenet(net_size, block)
model = NearSurroundTransfer(base_net, trans_out_size=trans_out_shape[2],
        trans_out_channels=trans_out_shape[1], conv_out_channels=chan,
        out_neurons=n_neurons, iterations=6, downscale=1, conv_k=ff_k,
        r_conv_k=r_k, conv_groups=groups, dilation=dilation).cpu()
state_dict = torch.load(model_dir + 'model.pt')
test_corrs = np.load(model_dir + 'test_corrs.npy')
model.load_state_dict(state_dict)
print(f'model loaded, with average time-bin correlation {np.mean(test_corrs)}')

model loaded, with average time-bin correlation 0.5845137781042127


In [7]:
def get_model_resp(model_this, stimuli):
    return model_this.nonlin(model_this.ff_conv(model_this.base(stimuli)))

In [8]:
def get_self_weights_fn(model):
    weights = model.r_conv.weight.data.numpy()
    assert weights.shape[0] == weights.shape[1]
    self_weights = [weights[i, i, :, :] for i in range(weights.shape[0])]
    return self_weights

In [9]:
from thesis_v2.analysis.hal_analysis_refactor.model_orientation_tuning import (
    model_orientation_tuning_one,
    get_bars,
    get_stimuli_dict
)

In [10]:
model_orientation_tuning_one(
    model=model, get_resp_fn=get_model_resp, get_self_weights_fn=get_self_weights_fn,
    bars=get_bars(),
    stimuli_dict=get_stimuli_dict(num_channel=3,normalize=True)
)

1.145158 0.7135249 (320, 32, 20, 20)


{'goods': array([-0.61461494,  0.53862978,  0.664997  ,  0.84330563,  0.64094529,
         0.91505362, -0.08598196,  0.43265851,  0.2821949 ,  0.66228359,
        -0.66971476, -0.42897534,  0.73764416,  0.52934442,  0.22389294,
         0.38395138,  0.13140208,  0.31369837, -0.10801406,  0.65595838,
         0.82118994,  0.33163086, -0.01586163,  0.72442623,  0.80276275,
        -0.3438378 ,  0.03580646,  0.62995051, -0.26724021, -0.34661706,
        -0.07160327,  0.644158  ]),
 'bads': array([-0.55361107,  0.65578551,  0.52336873,  0.60797159,  0.14779641,
         0.28528323, -0.27566151,  0.48296849, -0.12962201,  0.7068875 ,
        -0.48632414, -0.24529913,  0.4906124 ,  0.42646197,  0.57524699,
         0.65141551,  0.42671357,  0.49246707, -0.23462235,  0.73119276,
         0.57634038,  0.63175146,  0.22746946,  0.15599237,  0.69112841,
         0.14743605,  0.03281135,  0.64301955, -0.38283947, -0.40439524,
         0.02984284,  0.63101131]),
 'diffs': array([-0.06100387, -0.11

with some small change of Hal's code I get the following

```
Singularity yimeng-thesis-v2-20200106_2c0c603d8a871cd40d99848371ad443a.simg:/my_data/gaya-data/analysis/scripts> python model_orientation_tuning.py
model loaded, with average time-bin correlation 0.5845137781042127
[-0.06100387 -0.11715573  0.14162827  0.23533404  0.49314889  0.62977038
  0.18967954 -0.05030998  0.41181691 -0.04460391 -0.18339062 -0.18367621
  0.24703176  0.10288245 -0.35135405 -0.26746413 -0.2953115  -0.17876871
  0.12660829 -0.07523438  0.24484956 -0.3001206  -0.2433311   0.56843386
  0.11163433 -0.49127385  0.00299511 -0.01306904  0.11559926  0.05777818
 -0.10144611  0.01314669]
average diff between preferred and orthogonal direciton is 0.022963242337169763
standard error of that diff is 0.04724510368398206
--------------
average diff across all models is 0.022963242337169763
SE across models is 0.04724510368398206
--------------
Traceback (most recent call last):
  File "model_orientation_tuning.py", line 274, in <module>
    r, p = pearsonr(all_diffs.mean(0), all_perfs)
  File "/opt/conda/envs/leelab/lib/python3.7/site-packages/scipy/stats/stats.py", line 3501, in pearsonr
    raise ValueError('x and y must have length at least 2.')
ValueError: x and y must have length at least 2.
Singularity yimeng-thesis-v2-20200106_2c0c603d8a871cd40d99848371ad443a.simg:/my_data/gaya-data/analysis/scripts> python
```

In [14]:
a = [-0.06100387, -0.11715573,  0.14162827,  0.23533404,  0.49314889,
         0.62977038,  0.18967954, -0.05030998,  0.41181691, -0.04460391,
        -0.18339062, -0.18367621,  0.24703176,  0.10288245, -0.35135405,
        -0.26746413, -0.2953115 , -0.17876871,  0.12660829, -0.07523438,
         0.24484956, -0.3001206 , -0.2433311 ,  0.56843386,  0.11163433,
        -0.49127385,  0.00299511, -0.01306904,  0.11559926,  0.05777818,
        -0.10144611,  0.01314669]

In [15]:
b=[-0.06100387, -0.11715573 , 0.14162827,  0.23533404,  0.49314889,  0.62977038,
  0.18967954, -0.05030998,  0.41181691, -0.04460391, -0.18339062, -0.18367621,
  0.24703176,  0.10288245, -0.35135405, -0.26746413, -0.2953115,  -0.17876871,
  0.12660829, -0.07523438,  0.24484956, -0.3001206,  -0.2433311,   0.56843386,
  0.11163433, -0.49127385,  0.00299511, -0.01306904,  0.11559926,  0.05777818,
 -0.10144611,  0.01314669]

In [16]:
np.array(a)-np.array(b)

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [ ]:
# actually there is a bug in hal's code. no BN!!!

In [17]:
model = NearSurroundTransfer(base_net, trans_out_size=trans_out_shape[2],
        trans_out_channels=trans_out_shape[1], conv_out_channels=chan,
        out_neurons=n_neurons, iterations=6, downscale=1, conv_k=ff_k,
        r_conv_k=r_k, conv_groups=groups, dilation=dilation).cpu()
state_dict = torch.load(model_dir + 'model.pt')
test_corrs = np.load(model_dir + 'test_corrs.npy')
model.load_state_dict(state_dict)
model.eval()
model_orientation_tuning_one(
    model=model, get_resp_fn=get_model_resp, get_self_weights_fn=get_self_weights_fn,
    bars=get_bars(),
    stimuli_dict=get_stimuli_dict(num_channel=3,normalize=True)
)

1.1458682 0.8195893 (320, 32, 20, 20)


{'goods': array([-0.61461494,  0.53862978,  0.664997  ,  0.84330563,  0.64094529,
         0.91505362, -0.17936045,  0.27547381,  0.2821949 ,  0.66228359,
        -0.66971476, -0.24529913,  0.73764416,  0.52934442,  0.22389294,
         0.38395138,  0.13140208,  0.75872639, -0.10801406,  0.60919962,
         0.82118994,  0.40513735, -0.01586163,  0.72442623,  0.80276275,
        -0.3438378 ,  0.03580646,  0.62995051, -0.26724021, -0.26207352,
        -0.07160327,  0.644158  ]),
 'bads': array([-0.55361107,  0.65578551,  0.52336873,  0.60797159,  0.14779641,
         0.28528323, -0.24597069,  0.64015319, -0.12962201,  0.7068875 ,
        -0.48632414, -0.42897534,  0.4906124 ,  0.42646197,  0.57524699,
         0.65141551,  0.42671357,  0.6663222 , -0.23462235,  0.63838455,
         0.57634038,  0.44662822,  0.22746946,  0.15599237,  0.69112841,
         0.14743605,  0.03281135,  0.64301955, -0.38283947, -0.48893879,
         0.02984284,  0.63101131]),
 'diffs': array([-0.06100387, -0.11

In [18]:
a_fixed = [-0.06100387, -0.11715573,  0.14162827,  0.23533404,  0.49314889,
         0.62977038,  0.06661025, -0.36467937,  0.41181691, -0.04460391,
        -0.18339062,  0.18367621,  0.24703176,  0.10288245, -0.35135405,
        -0.26746413, -0.2953115 ,  0.09240419,  0.12660829, -0.02918493,
         0.24484956, -0.04149087, -0.2433311 ,  0.56843386,  0.11163433,
        -0.49127385,  0.00299511, -0.01306904,  0.11559926,  0.22686527,
        -0.10144611,  0.01314669]

In [19]:
np.array(a)-np.array(a_fixed)

array([ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.12306929,  0.31436939,  0.        ,  0.        ,
        0.        , -0.36735242,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        , -0.2711729 ,  0.        , -0.04604945,
        0.        , -0.25862973,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        , -0.16908709,
        0.        ,  0.        ])